In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from gsfanalysis.statistics import mode as cxxmode

In [3]:
def histmode(data, tol=0.0001, max_iter=10):
    mask = np.ones(len(data), dtype=np.bool)
    
    for _ in range(max_iter):
        hist, bins = np.histogram(data[mask])
        i = np.argmax(hist)
        mode = 0.5 * (bins[i] + bins[i+1])
        
        if bins[i+1] - bins[i] <= tol or len(bins) == 2:
            return mode
        
        mask = (data > bins[max(0,i-1)]) & (data < bins[min(i+2, len(bins)-1)])
        
    #print("Warning: mode accuracy is only: ", bins[i+1] - bins[i])
    return mode

In [4]:
def pymode(x):
    #x = np.sort(x)
    #print("len",len(x),"interval:",x[0],x[-1])
    if len(x) == 1:
        return x[0]
    elif len(x) == 2:
        return 0.5*(x[0] + x[1])
    elif len(x) == 3:
        if x[2] - x[1] < x[1] - x[0]:
            return 0.5*(x[2] + x[1])
        elif x[2] - x[1] > x[1] - x[0]:
            return 0.5*(x[1] + x[0])
        else:
            return x[1]
    else:
        wmin = x[-1] - x[0]
        
        if wmin == 0.0:
            return x[0]
        
        #print("wmin",wmin,"xrange",x[-1], x[0])
        N = math.ceil(len(x)//2)

        for i in range(len(x)-N):
            w = x[i+N-1] - x[i]
            if w < wmin:
                wmin = w
                j = i

        return pymode(x[j:j+N-1])

In [5]:
def sample(n):
    w = np.random.uniform(0,1,n) > 0.5
    
    data1 = np.random.normal(0,5,n)
    data2 = np.random.normal(4,2,n)
    
    return np.concatenate([data1[w], data2[~w]])

In [6]:
data = np.sort(sample(10000))

In [7]:
import timeit

In [17]:
n = 100000
time_cxx = timeit.timeit("cxxmode(data, is_sorted=True)", globals=globals(), number=n)
time_cxx, time_cxx / n

(0.9494383660021413, 9.494383660021412e-06)

In [18]:
time_cxx = time_cxx / n

In [19]:
n = 1000
time_py = timeit.timeit("pymode(data)", globals=globals(), number=n)/n
time_py

0.002271482599997398

In [20]:
n = 1000
time_histmode = timeit.timeit("histmode(data)", globals=globals(), number=n) / n
time_histmode

/tmp/ipykernel_71148/1798550146.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.ones(len(data), dtype=np.bool)


0.0008940155519994732

In [21]:
time_py / time_cxx

239.24487163522477

In [22]:
time_histmode / time_cxx

94.16256852606028

In [14]:
def plot(data):
    cxxmode = mode(data)
    hwm = half_width_mode(data)
    _ = plt.hist(data, bins=300)
    plt.vlines([cxxmode], ymin=0, ymax=plt.gca().get_ylim()[1], color='black', label='cxx mode {:.6f}'.format(cxxmode))
    plt.vlines([hwm], ymin=0, ymax=plt.gca().get_ylim()[1], color='red', label='half width mode {:.6f}'.format(hwm))
    plt.legend()

In [15]:
#plot(np.sort(sample(10000)))

In [16]:
#plot(np.sort(np.random.normal(3.5, 2, 100000)))